In [21]:
import numpy as np

#from assets.keras.keras.datasets import shapes_3d
from assets.keras.keras.preprocessing.image import ImageDataGenerator
from assets.keras.keras.models import Sequential
from assets.keras.keras.layers.core import Dense, Dropout, Activation, Flatten
from assets.keras.keras.layers.convolutional import Convolution3D, MaxPooling3D, ZeroPadding3D
from assets.keras.keras.optimizers import SGD, Adam
from assets.keras.keras.utils import np_utils, generic_utils
from assets.keras.keras.regularizers import l2
import theano
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

#load dataset from gen_dataset
import cPickle as pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
from assets.gen_motion import gen_dataset
import math
#gen_dataset(size, N, obj_type, obj_theta, obj_vel, types, velocity, theta, coherence, dots)
X_train1,Y_train1,X_test1,Y_test1 = gen_dataset((16,32,32),20,'square', (5,5,0), 0, 1,['translate'],[1],[0],[0],[150],[0])
X_train2,Y_train2,X_test2,Y_test2 = gen_dataset((16,32,32),20,'circle', (5,5,0), 0, 1,['translate'],[1],[0],[0],[150],[0])

X_train = np.concatenate((X_train1,X_train2),axis=0)
Y_train = np.concatenate((Y_train1,Y_train2))
X_test = np.concatenate((X_test1,X_test2))
Y_test = np.concatenate((Y_test1,Y_test2))

print 'Train data shape: ', X_train.shape
print 'Train labels shape: ', Y_train.shape
print 'Test data shape: ', X_test.shape
print 'Test labels shape: ', Y_test.shape

###
#X_train = X_train[0:200,:,:,:,:]
#Y_train = Y_train[0:200]
#X_val = X_val[0:40,:,:,:,:]
#Y_val = Y_val[0:40]
# convert class vectors to binary class matrices

#Y_train/=math.pi
#Y_test/=math.pi
Y_train = np_utils.to_categorical(Y_train[:,6]-1, 2)
Y_test = np_utils.to_categorical(Y_test[:,6]-1, 2)

Train data shape:  (36, 1, 16, 32, 32)
Train labels shape:  (36, 7)
Test data shape:  (4, 1, 16, 32, 32)
Test labels shape:  (4, 7)


In [18]:

print('X_train shape:', X_train.shape)
print('Y_train shape:', Y_train.shape)
print('X_test shape:', X_test.shape)
print('Y_test shape:', Y_test.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

('X_train shape:', (36, 1, 16, 32, 32))
('Y_train shape:', (36, 2))
('X_test shape:', (4, 1, 16, 32, 32))
('Y_test shape:', (4, 2))
(36, 'train samples')
(4, 'test samples')


In [ ]:

# CNN Training parameters
batch_size = 20
nb_classes = 2
nb_epoch = 200


# number of convolutional filters to use at each layer
nb_filters = [2,2,2]

# level of pooling to perform at each layer (POOL x POOL)
nb_pool = [2,2,2]

# level of convolution to perform at each layer (CONV x CONV)
nb_conv = [3,3,3]

# Regularization
reg = 1e-4

model = Sequential()
#model.add(ZeroPadding3D((1,1,1),))
model.add(Convolution3D(nb_filters[0],len_conv_dim1=nb_conv[0], len_conv_dim2=nb_conv[0], len_conv_dim3=nb_conv[0], border_mode='valid',
                         activation='relu', W_regularizer=l2(reg),dim_ordering='th',input_shape=(1,16,32,32)))
model.add(MaxPooling3D(pool_size=(1, nb_pool[0], nb_pool[0])))
#model.add(Dropout(0.5))
#model.add(ZeroPadding3D((1,1,1)))
model.add(Convolution3D(nb_filters[1],len_conv_dim1=nb_conv[1], len_conv_dim2=nb_conv[1], len_conv_dim3=nb_conv[1], border_mode='valid',
                        activation='relu', W_regularizer=l2(reg)))
#model.add(MaxPooling3D(pool_size=(nb_pool[1], nb_pool[1], nb_pool[1])))
#model.add(Dropout(0.5))
#model.add(ZeroPadding3D((1,1,1)))
model.add(Convolution3D(nb_filters[2],len_conv_dim1=nb_conv[1], len_conv_dim2=nb_conv[1], len_conv_dim3=nb_conv[1], border_mode='valid',
                        activation='relu', W_regularizer=l2(reg)))
model.add(Flatten())
#model.add(Dropout(0.5))
model.add(Dense(4, init='normal', activation='relu', W_regularizer=l2(reg)))
model.add(Dense(nb_classes, init='normal', W_regularizer=l2(reg)))
model.add(Activation('softmax'))

#Adam
sgd = Adam(lr=1e-3)#, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mse', optimizer=sgd)

hist = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, show_accuracy=True, verbose=1,
          validation_split=.1)
scoretr = model.evaluate(X_train, Y_train, batch_size=batch_size, show_accuracy=True)
#scoreva = model.evaluate(X_val, Y_val, batch_size=batch_size, show_accuracy=True)
scorete = model.evaluate(X_test, Y_test, batch_size=batch_size, show_accuracy=True)
print('Train loss:', scoretr[0])
print('Train accuracy:', scoretr[1])
#print('Val loss:', scoreva[0])
#print('Val accuracy:', scoreva[1])
print('Test loss:', scorete[0])
print('Test accuracy:', scorete[1])

Train on 32 samples, validate on 4 samples
Epoch 1/200
32/32 [==============================] - 0s - loss: 0.4464 - acc: 0.5000 - val_loss: 0.3869 - val_acc: 0.5000
Epoch 2/200
32/32 [==============================] - 0s - loss: 0.3180 - acc: 0.6250 - val_loss: 0.5175 - val_acc: 0.5000
Epoch 3/200
32/32 [==============================] - 0s - loss: 0.2952 - acc: 0.6250 - val_loss: 0.5565 - val_acc: 0.5000
Epoch 4/200
32/32 [==============================] - 0s - loss: 0.2684 - acc: 0.6562 - val_loss: 0.5634 - val_acc: 0.2500
Epoch 5/200
32/32 [==============================] - 0s - loss: 0.2505 - acc: 0.7188 - val_loss: 0.5636 - val_acc: 0.2500
Epoch 6/200
32/32 [==============================] - 0s - loss: 0.2469 - acc: 0.7812 - val_loss: 0.5638 - val_acc: 0.2500
Epoch 7/200
32/32 [==============================] - 0s - loss: 0.2448 - acc: 0.7812 - val_loss: 0.5640 - val_acc: 0.2500
Epoch 8/200
32/32 [==============================] - 0s - loss: 0.2549 - acc: 0.7500 - val_loss: 0.5640